# 1. Preparation

## 1.1 Prepare for LLM

In [ ]:
# %pip install llama-index-llms-azure-openai
# %pip install llama-index-graph-stores-nebula
# %pip install llama-index-llms-openai
# %pip install llama-index-embeddings-azure-openai

In [1]:
# For OpenAI

import os

# os.environ["OPENAI_API_KEY"] = "INSERT YOUR KEY"

import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output

from llama_index.core import (
    KnowledgeGraphIndex,
    VectorStoreIndex,
    ServiceContext,
    SimpleDirectoryReader,
    StorageContext,
    PromptTemplate,
    load_index_from_storage
)
from llama_index.core.query_engine import KnowledgeGraphQueryEngine

from llama_index.graph_stores.nebula import NebulaGraphStore

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from IPython.display import Markdown, display

from finllmqa.api.core import LLM_API_URL

from llama_index.core import Settings

llm = OpenAI(model="gpt-3.5-turbo", api_base=LLM_API_URL, api_key='null')
embed_model = OpenAIEmbedding(api_base=LLM_API_URL, api_key='null')

Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
# For Azure OpenAI

import os
import json
import openai
from langchain.embeddings import OpenAIEmbeddings
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    KnowledgeGraphIndex,
    ServiceContext
)
from llama_index.core import set_global_service_context

from llama_index.core.storage.storage_context import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore

import logging
import sys

from IPython.display import Markdown, display

from finllmqa.api.core import LLM_API_URL

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

openai.api_type = "azure"
openai.api_base = LLM_API_URL
openai.api_version = "2024-03-01"
# os.environ["OPENAI_API_KEY"] = "youcannottellanyone"
# openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = 'null'

llm = AzureOpenAI(
    engine="<foo-bar-deployment>",
    temperature=0,
    openai_api_version=openai.api_version,
    model_kwargs={
        "api_key": openai.api_key,
        "api_base": openai.api_base,
        "api_type": openai.api_type,
        "api_version": openai.api_version,
    },
)

# You need to deploy your own embedding model as well as your own chat completion model
embedding_llm = LangchainEmbedding(
    OpenAIEmbeddings(
        model="text-embedding-ada-002",
        deployment="<foo-bar-deployment>",
        openai_api_key=openai.api_key,
        openai_api_base=openai.api_base,
        openai_api_type=openai.api_type,
        openai_api_version=openai.api_version,
    ),
    embed_batch_size=1,
)

# service_context = ServiceContext.from_defaults(
#     llm=llm,
#     embed_model=embedding_llm,
# )

# set_global_service_context(service_context)

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embedding_llm

## 1.2. Prepare for NebulaGraph as Graph Store


❗Access NebulaGraph Console to **create space** and **graph schema**

```sql
CREATE SPACE guardians(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);
:sleep 10;
USE guardians;
CREATE TAG entity(name string);
CREATE EDGE relationship(relationship string);
:sleep 10;
CREATE TAG INDEX entity_index ON entity(name(256));
```

In [ ]:
# %pip install nebula3-python ipython-ngql

In [6]:
os.environ['NEBULA_USER'] = "root"
os.environ['NEBULA_PASSWORD'] = "nebula" # default password
os.environ['NEBULA_ADDRESS'] = "127.0.0.1:9669" 

## 2. Load from disk Llama Indexes

**You have to run cells in $2 and $3 or download index.zip first**

Both the `KnowledgeGraphIndex` and `VectorStoreIndex` will be created only once, afterwards, we could persist their in-memory context to enable their reuse from disk anytime.

In [ ]:
from llama_index.core import load_index_from_storage

assert os.path.exists(os.path.join(os.path.abspath(os.path.join('..')), 'storage/storage_graph')), 'Do not have graph storage_context in disk'
assert os.path.exists(os.path.join(os.path.abspath(os.path.join('..')), 'storage/storage_vector')), 'Do not have vector storage_context in disk'

entries = os.listdir()
folders = [entry for entry in entries if os.path.isdir(os.path.join(entry))]

kg_index_ls = []
vector_index_ls = []
for nodes_group in folders:
    space_name = f"books_content_{nodes_group}"
    edge_types, rel_prop_names = ["relationship"], ["relationship"] # default, could be omit if create from an empty kg
    tags = ["entity"] # default, could be omit if create from an empty kg

    graph_store = NebulaGraphStore(
        space_name=space_name,
        edge_types=edge_types,
        rel_prop_names=rel_prop_names,
        tags=tags,
    )
    storage_context = StorageContext.from_defaults(persist_dir=f'../storage/storage_graph/{nodes_group}', graph_store=graph_store)
    kg_index = load_index_from_storage(
        storage_context=storage_context,
        space_name=space_name,
        edge_types=edge_types,
        rel_prop_names=rel_prop_names,
        tags=tags,
        include_embeddings=True,
    )
    kg_index_ls.append(kg_index)

    storage_context_vector = StorageContext.from_defaults(persist_dir=f'../storage_vector/{nodes_group}')
    vector_index = load_index_from_storage(
    #     service_context=service_context,
        storage_context=storage_context_vector
    )
    vector_index_ls.append(vector_index)

## 3. Prepare for different query approaches

We will do 4 types of query approaches with LLM, KG, VectorDB:

| QueryEngine | Knowledge Graph query engine                                 | Graph RAG query engine                                       | Vector RAG query engine                                      | Graph Vector RAG query engine                                |
| ----------- | ------------------------------------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ |
| Mechanism   | 1. **Text-to-GraphQuery** based on KG<br />2. Query KG with the result<br />3. Answer synthesis based on query result | 1. Get related entities of the question<br />2. Get n-depth **SubGraphs** of related entities from KG<br />3. Answer synthesis based on related SubGraphs | 1. Create embedding of question<br />2. Semantic search **top-k related doc chunks**<br />3. Answer synthesis based on related doc chunks | 1. Do retrieval as Vector and Graph RAG <br />2. Answer synthesis based on **both related chunks and SubGraphs** |


### 3.1 text-to-NebulaGraphCypher

Text-to-NebulaGraphCypher approach Translate task/question into a Graph Cypher Query, and answer based on its query result.

In [ ]:
from llama_index.core.query_engine import KnowledgeGraphQueryEngine

from llama_index.core import load_index_from_storage

assert os.path.exists(os.path.join(os.path.abspath(os.path.join('..')), 'storage/storage_graph')), 'Do not have graph storage_context in disk'
assert os.path.exists(os.path.join(os.path.abspath(os.path.join('..')), 'storage/storage_vector')), 'Do not have vector storage_context in disk'

entries = os.listdir()
folders = [entry for entry in entries if os.path.isdir(os.path.join(entry))]

nl2kg_qg_ls = []
for nodes_group in folders:
    space_name = f"books_content_{nodes_group}"
    edge_types, rel_prop_names = ["relationship"], ["relationship"] # default, could be omit if create from an empty kg
    tags = ["entity"] # default, could be omit if create from an empty kg

    graph_store = NebulaGraphStore(
        space_name=space_name,
        edge_types=edge_types,
        rel_prop_names=rel_prop_names,
        tags=tags,
    )
    storage_context = StorageContext.from_defaults(persist_dir=f'../storage/storage_graph/{nodes_group}', graph_store=graph_store)

    nl2kg_query_engine = KnowledgeGraphQueryEngine(
        storage_context=storage_context,
    #     service_context=service_context,
        verbose=True
    )
    nl2kg_qg_ls.append(nl2kg_query_engine)

In [ ]:
nl2kg_query_engine.get_prompts()

### 3.2 Graph RAG query engine

Graph RAG takes SubGraphs related to entities of the task/question as Context.

```
           Graph RAG with Llama Index
                  ┌────┬────┬────┬────┐                  
                  │ 1  │ 2  │ 3  │ 4  │                  
                  ├────┴────┴────┴────┤                  
                  │  Docs/Knowledge   │                  
┌───────┐         │        ...        │       ┌─────────┐
│       │         ├────┬────┬────┬────┤       │         │
│       │         │ 95 │ 96 │    │    │       │         │
│       │         └────┴────┴────┴────┘       │         │
│ User  │─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─▶   LLM   │
│       │                                     │         │
│       │                                     │         │
└───────┘    ┌ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ┐  └─────────┘
    │          ┌──────────────────────────┐        ▲     
    └────────┼▶│  Tell me about x, please │├───────┘     
               └──────────────────────────┘              
             │ Below are knowledge about x │             
               x->y<-z,x->h->i, m<-n,...                            
             │ Please answer based on them │             
              ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
```

In [ ]:
kg_rag_qg_ls = []
for kg_index in kg_index_ls:
    kg_rag_query_engine = kg_index.as_query_engine(
        include_text=False,
        retriever_mode="hybrid",
        response_mode="tree_summarize",
    )
    kg_rag_qg_ls.append(kg_rag_query_engine)

### 3.3 Vector RAG query engine

Vector RAG is the common approach to find topK semantic related doc chunks as context to synthesize the answer.

```
                  RAG with Llama Index
                  ┌────┬────┬────┬────┐                  
                  │ 1  │ 2  │ 3  │ 4  │                  
                  ├────┴────┴────┴────┤                  
                  │  Docs/Knowledge   │                  
┌───────┐         │        ...        │       ┌─────────┐
│       │         ├────┬────┬────┬────┤       │         │
│       │         │ 95 │ 96 │    │    │       │         │
│       │         └────┴────┴────┴────┘       │         │
│ User  │─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─▶   LLM   │
│       │                                     │         │
│       │                                     │         │
└───────┘    ┌ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ┐  └─────────┘
    │          ┌──────────────────────────┐        ▲     
    └────────┼▶│  Tell me ....., please   │├───────┘     
               └──────────────────────────┘              
             │ ┌────┐ ┌────┐               │             
               │ 3  │ │ 96 │                             
             │ └────┘ └────┘               │             
              ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
```

In [ ]:
vector_rag_qg_ls = []
for vetor_index in vector_index_ls:
    vector_rag_query_engine = vector_index.as_query_engine()
    vector_rag_qg_ls.append(vector_rag_query_engine)

### 3.4 Graph+Vector RAG query engine

This is a combined Graph+Vector Based RAG, where we will retrieve both VectorDB and KG SubGraphs as the context, for synthesis of the answer.

```
           Graph + Vector RAG with Llama Index
                  ┌────┬────┬────┬────┐                  
                  │ 1  │ 2  │ 3  │ 4  │                  
                  ├────┴────┴────┴────┤                  
                  │  Docs/Knowledge   │                  
┌───────┐         │        ...        │       ┌─────────┐
│       │         ├────┬────┬────┬────┤       │         │
│       │         │ 95 │ 96 │    │    │       │         │
│       │         └────┴────┴────┴────┘       │         │
│ User  │─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─▶   LLM   │
│       │                                     │         │
│       │                                     │         │
└───────┘    ┌ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ┐  └─────────┘
    │          ┌──────────────────────────┐        ▲     
    └────────┼▶│  Tell me ....., please   │├───────┘     
               └──────────────────────────┘              
             │ ┌────┐┌────┐               │             
               │ 3  ││ 96 │ x->y<-z,x->h...                            
             │ └────┘└────┘               │             
              ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
```

To implement that in Llama Index, we create a `CustomRetriever` to comebine the two: 

In [3]:
# import QueryBundle
from llama_index.core import QueryBundle

# import NodeWithScore
from llama_index.core.schema import NodeWithScore

# Retrievers
from llama_index.core.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever

from typing import List


class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both Vector search and Knowledge Graph search"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        kg_retriever: KGTableRetriever,
        mode: str = "OR",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        kg_nodes = self._kg_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(kg_ids)
        else:
            retrieve_ids = vector_ids.union(kg_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

Next, we will create instances of the Vector and KG retrievers, which will be used in the instantiation of the Custom Retriever.

In [ ]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

kg_vec_rag_qg_ls = []
for kg_index, vector_index in zip(kg_index_ls, vector_index_ls):
    # create custom retriever
    vector_retriever = VectorIndexRetriever(index=vector_index)
    kg_retriever = KGTableRetriever(
        index=kg_index, retriever_mode="keyword", include_text=False
    )
    custom_retriever = CustomRetriever(vector_retriever, kg_retriever)

    # create response synthesizer
    response_synthesizer = get_response_synthesizer(
    #     service_context=service_context,
        response_mode="tree_summarize",
    )
    kg_vector_rag_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer
    )
    kg_vec_rag_qg_ls.append(kg_vector_rag_query_engine)

### 3.5 General load index from disk and get query engine function

In [4]:
from llama_index.core import QueryBundle
from llama_index.core.schema import NodeWithScore
from llama_index.core.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
def get_all_query_engine_from_cache_index(kg_index_folder_path, vector_index_folder_path, nodes_group: str|List[str]):
    if isinstance(nodes_group, str):
        nodes_group_ls = [nodes_group]
    else:
        nodes_group_ls = nodes_group
    query_engine_dc = {
        # 'nl2kg': {},
        'kg_rag': {},
        'vec_rag': {},
        'kg_vec_rag': {}
    }
    for nodes_group in nodes_group_ls:
        space_name = f"books_content_{nodes_group}"
        edge_types, rel_prop_names = ["relationship"], ["relationship"] # default, could be omit if create from an empty kg
        tags = ["entity"] # default, could be omit if create from an empty kg

        graph_store = NebulaGraphStore(
            space_name=space_name,
            edge_types=edge_types,
            rel_prop_names=rel_prop_names,
            tags=tags,
        )
        storage_context_kg = StorageContext.from_defaults(persist_dir=kg_index_folder_path + f'/{nodes_group}', graph_store=graph_store)
        kg_index = load_index_from_storage(
            storage_context=storage_context_kg,
            space_name=space_name,
            edge_types=edge_types,
            rel_prop_names=rel_prop_names,
            tags=tags,
            include_embeddings=True,
        )

        storage_context_vector = StorageContext.from_defaults(persist_dir=vector_index_folder_path + f'/{nodes_group}')
        vector_index = load_index_from_storage(
            storage_context=storage_context_vector
        )

        # # text2cypher query engine
        # nl2kg_query_engine = KnowledgeGraphQueryEngine(
        #     storage_context=storage_context_kg,
        #     verbose=True
        # )
        # query_engine_dc['nl2kg'].append(nl2kg_query_engine)
        
        # kg_rag query engine
        kg_rag_query_engine = kg_index.as_query_engine(
            include_text=False,
            response_mode="tree_summarize"
        )
        query_engine_dc['kg_rag'][nodes_group] = kg_rag_query_engine
        # vec_rag query engine
        vec_rag_query_engine = vector_index.as_query_engine(response_mode="tree_summarize")
        query_engine_dc['vec_rag'][nodes_group] =  vec_rag_query_engine
        # kg_vec_rag query engine
        vector_retriever = VectorIndexRetriever(index=vector_index)
        kg_retriever = KGTableRetriever(
            index=kg_index, retriever_mode="keyword", include_text=False
        )
        custom_retriever = CustomRetriever(vector_retriever, kg_retriever)

        response_synthesizer = get_response_synthesizer(response_mode="tree_summarize")
        kg_vector_rag_query_engine = RetrieverQueryEngine(
            retriever=custom_retriever,
            response_synthesizer=response_synthesizer
        )
        query_engine_dc['kg_vec_rag'][nodes_group] = kg_vector_rag_query_engine
    return query_engine_dc


## 6. Base Query with all the Engines

### 6.1 Text-to-GraphQuery

In [ ]:
response_nl2kg = nl2kg_query_engine.query("什么是经济学十大原理.")


display(Markdown(f"<b>{response_nl2kg}</b>"))

# Cypher:

print("Cypher Query:")

graph_query = nl2kg_query_engine.generate_query(
    "什么是经济学十大原理",
)
graph_query = graph_query.replace("WHERE", "\n  WHERE").replace("RETURN", "\nRETURN")

display(
    Markdown(
        f"""
```cypher
{graph_query}
```
"""
    )
)

### 6.2 Graph RAG

In [ ]:
response_graph_rag = kg_rag_query_engine.query("什么是经济学十大原理")

display(Markdown(f"<b>{response_graph_rag}</b>"))

### 6.3 Vector RAG

In [ ]:
response_vector_rag = vector_rag_query_engine.query("什么是经济学十大原理")

display(Markdown(f"<b>{response_vector_rag}</b>"))

In [ ]:
display(
    Markdown(
        llm.complete(f"""
Compare the two QA result on "什么是经济学十大原理", list the differences between them, to help evalute them. Output in markdown table.

Result from Graph: {response_graph_rag}
---
Result from Vector: {response_vector_rag}

"""
           ).text
    )
)

### 6.4 Graph + Vector RAG

In [ ]:
response_graph_vector_rag = graph_vector_rag_query_engine.query("什么是经济学十大原理")

display(Markdown(f"<b>{response_graph_vector_rag}</b>"))

### 6.5 Overall Comparision

Let's compare the results of them.

First check the information that were coverred by different approaches:

In [ ]:
display(
    Markdown(
        llm(f"""
Compare the QA results on "Tell me about Peter Quill.", list the knowledge facts between them, to help evalute them. Output in markdown table.

Result text2GraphQuery: {response_nl2kg}
---
Result Graph: {response_graph_rag}
---
Result Vector: {response_vector_rag}
---
Result Graph+Vector: {response_graph_vector_rag}
---

"""
           )
    )
)

**Conclusion**

- The pure **KG**(both text2GraphQuery and Graph RAG) comes with **concise** results, and much **lower cost**(for cost comparision see our previous result [here](https://gpt-index.readthedocs.io/en/latest/examples/index_structs/knowledge_graph/KnowledgeGraphIndex_vs_VectorStoreIndex_vs_CustomIndex_combined.html#comparison-of-results) )
- The **Graph+Vector** RAG could be more **comprehensive** in case the question envolves knowledge that's fine-grained **spread** across more chunks than top-K searching.


| QueryEngine | Knowledge Graph query engine                                 | Graph RAG query engine                                       | Vector RAG query engine                                      | Graph Vector RAG query engine                                |
| ----------- | ------------------------------------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ |
| Mechanism   | 1. **Text-to-GraphQuery** based on KG<br />2. Query KG with the result<br />3. Answer synthesis based on query result | 1. Get related entities of the question<br />2. Get n-depth **SubGraphs** of related entities from KG<br />3. Answer synthesis based on related SubGraphs | 1. Create embedding of question<br />2. Semantic search **top-k related doc chunks**<br />3. Answer synthesis based on related doc chunks | 1. Do retrieval as Vector and Graph RAG <br />2. Answer synthesis based on **both related chunks and SubGraphs** |
| Performance | Concise                                                      | Concise                                                      | Fruitful                                                     | Fruitful, could be more comprehensive                        |
| Cost        | Low                                                          | Low                                                          | High                                                         | High                                                         |


**Conclusion**

For those tasks:

- Potentially cares more relationed knowledge
- Schema of the KG is sophisticated to be hard for text2cypher to express the task
- KG quality isn't good enough
- Multiple "starting entities" are involved

Graph RAG could be a better approach to start with.

## 7. Financial Evaluation on four types of engines

### 7.1 FinEval on query engines base on nodes of different chunk sizes and chunk overlaps

In [9]:
chunk_size_ls = [256, 512, 1024]
chunk_overlap_pct_ls = [1/8, 1/4]
nodes_group_ls = []
for chunk_size in chunk_size_ls:
    for chunk_overlap_pct in chunk_overlap_pct_ls:
        chunk_overlap = int(chunk_size * chunk_overlap_pct)
        nodes_group = f'size_{chunk_size}_overlap_{chunk_overlap}'
        nodes_group_ls.append(nodes_group)
query_engine_dc = get_all_query_engine_from_cache_index(kg_index_folder_path='../storage/storage_graph',
                                                        vector_index_folder_path='../storage/storage_vector',
                                                        nodes_group=nodes_group_ls)

INFO:llama_index.core.indices.loading:Loading all indices.
INFO:llama_index.core.indices.loading:Loading all indices.
INFO:llama_index.core.indices.loading:Loading all indices.


Exception ignored in: <function NebulaGraphStore.__del__ at 0x000001CE40D5AD40>
Traceback (most recent call last):
  File "e:\anaconda\app\envs\chatglm3\lib\site-packages\llama_index\graph_stores\nebula\base.py", line 242, in __del__
    self._session_pool.close()
AttributeError: 'NoneType' object has no attribute 'close'


INFO:llama_index.core.indices.loading:Loading all indices.
INFO:llama_index.core.indices.loading:Loading all indices.
INFO:llama_index.core.indices.loading:Loading all indices.
INFO:llama_index.core.indices.loading:Loading all indices.
INFO:llama_index.core.indices.loading:Loading all indices.
INFO:llama_index.core.indices.loading:Loading all indices.
INFO:llama_index.core.indices.loading:Loading all indices.
INFO:llama_index.core.indices.loading:Loading all indices.
INFO:llama_index.core.indices.loading:Loading all indices.


In [15]:
import os
import pandas as pd
import json
import time

choices = ["A", "B", "C", "D"]
eval_path = ''

def fineval(args, evaluator, take):
    assert os.path.exists(eval_path + "subject_mapping.json"), "subject_mapping.json not found!"
    with open(eval_path+ "subject_mapping.json") as f:
        subject_mapping = json.load(f)
    subject_mapping = {'microeconomics': subject_mapping['microeconomics']}
    filenames = os.listdir(eval_path + "data/val")
    subject_list = [val_file.replace("_val.csv", "") for val_file in filenames]
    accuracy, summary = {}, {}

    run_date = time.strftime('%Y-%m-%d_%H-%M-%S', time.localtime(time.time()))
    output_dir = args['output_dir']
    save_result_dir = os.path.join(output_dir, f"take{take}")
    if not os.path.exists(save_result_dir):
        os.makedirs(save_result_dir, exist_ok=True)

    print(f"############# nodes group: {args['nodes_group']} ###############")

    all_answers = {}
    for index, subject_name in enumerate(subject_list):
        print(
            f"{index / len(subject_list)} Inference starts at {run_date} on {args['model_name']} with subject of {subject_name}!")
        val_file_path = os.path.join('data/val', f'{subject_name}_val.csv')
        dev_file_path = os.path.join('data/dev', f'{subject_name}_dev.csv')
        test_file_path = os.path.join('data/test', f'{subject_name}_test.csv')

        val_df = pd.read_csv(val_file_path) if args['do_test'] is False else pd.read_csv(test_file_path)
        dev_df = pd.read_csv(dev_file_path) if args['few_shot'] else None

        correct_ratio, answers = evaluator.eval_subject(subject_name, val_df, dev_df,
                                                        save_result_dir=save_result_dir if args['do_save_csv'] else None,
                                                        few_shot=args['few_shot'],
                                                        cot=args['cot'],
                                                        )
        print(f"Subject: {subject_name}")
        print(f"Acc: {correct_ratio}")
        accuracy[subject_name] = correct_ratio
        summary[subject_name] = {"score": correct_ratio,
                                 "num": len(val_df),
                                 "correct": correct_ratio * len(val_df) / 100}
        all_answers[subject_name] = answers

    json.dump(all_answers, open(save_result_dir + '/submission.json', 'w'), ensure_ascii=False, indent=4)
    print("Accuracy:")
    for k, v in accuracy.items():
        print(k, ": ", v)

    total_num = 0
    total_correct = 0
    summary['grouped'] = {
        # "Accounting": {"correct": 0.0, "num": 0},
        # "Finance": {"correct": 0.0, "num": 0},
        "Economy": {"correct": 0.0, "num": 0},
        # "Certificate": {"correct": 0.0, "num": 0}
    }
    for subj, info in subject_mapping.items():
        group = info[2]
        summary['grouped'][group]["num"] += summary[subj]['num']
        summary['grouped'][group]["correct"] += summary[subj]['correct']
    for group, info in summary['grouped'].items():
        info['score'] = info["correct"] / info["num"]
        total_num += info["num"]
        total_correct += info["correct"]
    summary['All'] = {"score": total_correct / total_num, "num": total_num, "correct": total_correct}

    print('-' * 80)
    print("Accuracy_subject:")
    for k, v in accuracy.items():
        print(k, ": ", v)
    print('-' * 80)
    print("Accuracy_grouped:")
    for k, v in summary['grouped'].items():
        print(k, ": ", v['score'])

    print("Avg: ")
    print(summary['All']['score'])

    json.dump(summary, open(save_result_dir + '/summary.json', 'w'), ensure_ascii=False, indent=2)
    return summary

In [11]:
cot = False
few_shot = False
ntrain = 5
n_times = 1
do_save_csv = False
output_dir = eval_path + 'output'
model_name = 'chatglm'
do_test = False
args = dict(
    cot=cot,
    few_shot = few_shot,
    ntrain = ntrain,
    n_times = n_times,
    do_save_csv = do_save_csv,
    output_dir = output_dir,
    model_name = model_name,
    do_test = do_test
)

tree_summary_template = \
    "从不同来源获取的参考信息如下:\n" \
    "---------------------\n" \
    "{context_str}\n" \
    "---------------------\n" \
    "题目:{query_str}" 

graph_query_synthesis_prompt = \
    """
    你是一个专业的人工智能助手
    现在有一个基于Nebula搭建的财经百科知识图谱，给定这个图谱框架结构，请你根据框架结构将问题文本转化成Nebula Cypher查询语句
    保证查询语可以直接在Nebula终端中运行

    知识图谱框架:{schema}
    问题: {query_str}
    """

graph_response_answer_prompt = \
    """
    原问题被转化成了查询语句，查询语句和查询结果将作为参考信息，如下:

    查询语句: {kg_query_str}
    查询结果: {kg_response_str}
    题目: {query_str}
    """

In [21]:
from openai_evaluators import OpenAI_Evaluator
evaluator = OpenAI_Evaluator(choices=choices, api_key='null', k=args['ntrain'], model_name=args['model_name'])
benchmark_summary = fineval(args=args, evaluator=evaluator, take=1)
benchmark_summary

############# nodes group: size_1024_overlap_256 ###############
0.0 Inference starts at 2024-04-12_00-03-59 on chatglm with subject of microeconomics!


  0%|          | 0/40 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"


  2%|▎         | 1/40 [00:00<00:29,  1.33it/s]

INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"


...

INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"


 98%|█████████▊| 39/40 [00:24<00:00,  1.39it/s]

INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"


100%|██████████| 40/40 [00:25<00:00,  1.60it/s]

Subject: microeconomics
Acc: 0.0
Accuracy:
microeconomics :  0.0
--------------------------------------------------------------------------------
Accuracy_subject:
microeconomics :  0.0
--------------------------------------------------------------------------------
Accuracy_grouped:
Economy :  0.0
Avg: 
0.0


In [16]:
from query_engine_evaluator import QueryEngineEvaluator
kg_rag_summary_ls = []
for nodes_group in nodes_group_ls:
    args['nodes_group'] = nodes_group
    query_engine = query_engine_dc['kg_rag'][nodes_group]
    prompt_dict = dict(
        summary_template = [
            {
                'role': 'user',
                'content': tree_summary_template
             }])
    evaluator = QueryEngineEvaluator(query_engine=query_engine, prompt_dict=prompt_dict, choices=choices,
                                     k=args['ntrain'], model_name=args['model_name'])
    summary = fineval(args=args, evaluator=evaluator, take=1)
    kg_rag_summary_ls.append(summary)

############# nodes group: size_256_overlap_32 ###############
0.0 Inference starts at 2024-04-11_23-31-42 on chatglm with subject of microeconomics!


  0%|          | 0/40 [00:00<?, ?it/s]

{'response_synthesizer:summary_template': ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='你是一个中文人工智能助手，以下是中国关于microeconomics考试的单项选择题以及相应的参考内容，请根据参考信息选出其中的正确答案。', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='从不同来源获取的参考信息如下:\n---------------------\n{context_str}\n---------------------\n题目:{query_str}', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='选项:\nA. 平均总成本小于100\nB. 平均固定成本小于100\nC. 边际产量增加\nD. 平均可变成本小于100\n答案:\n', additional_kwargs={})])}
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1

  2%|▎         | 1/40 [00:10<06:51, 10.55s/it]

{'response_synthesizer:summary_template': ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='你是一个中文人工智能助手，以下是中国关于microeconomics考试的单项选择题以及相应的参考内容，请根据参考信息选出其中的正确答案。', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='从不同来源获取的参考信息如下:\n---------------------\n{context_str}\n---------------------\n题目:{query_str}', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='选项:\nA. 固定不变\nB. 经常变动\nC. 等于1\nD. 等于0\n答案:\n', additional_kwargs={})])}
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/

...

{'response_synthesizer:summary_template': ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='你是一个中文人工智能助手，以下是中国关于microeconomics考试的单项选择题以及相应的参考内容，请根据参考信息选出其中的正确答案。', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='从不同来源获取的参考信息如下:\n---------------------\n{context_str}\n---------------------\n题目:{query_str}', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='选项:\nA. 双方在每一期都选择“坦白”\nB. 双方在每一期都选择“抵赖”\nC. 双方在前$N-1$期都选择“坦白”，在最后一期都选择“抵赖”\nD. 双方在前$N-1$期都选择“抵赖”，在最后一期都选择“坦白”\n答案:\n', additional_kwargs={})])}
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP R

 98%|█████████▊| 39/40 [01:55<00:03,  3.17s/it]

{'response_synthesizer:summary_template': ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='你是一个中文人工智能助手，以下是中国关于microeconomics考试的单项选择题以及相应的参考内容，请根据参考信息选出其中的正确答案。', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='从不同来源获取的参考信息如下:\n---------------------\n{context_str}\n---------------------\n题目:{query_str}', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='选项:\nA. 收益\nB. 支付\nC. 决策\nD. 利润\n答案:\n', additional_kwargs={})])}
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 20

100%|██████████| 40/40 [01:57<00:00,  2.94s/it]

Subject: microeconomics
Acc: 42.5
Accuracy:
microeconomics :  42.5
--------------------------------------------------------------------------------
Accuracy_subject:
microeconomics :  42.5
--------------------------------------------------------------------------------
Accuracy_grouped:
Economy :  0.425
Avg: 
0.425
